## KNN (K-nearest neighbors) e uma visão inicial sobre problemas de classificação 

<small>Por: Hugo Soares, Cefas Rodrigues e Arilton Filho</small>

O KNN (K-nearest neighbors, <i> K-vizinhos mais próximos</i>) é um dos mais famosos e simples algoritmos de Machine Learning (ML). Ele é o pontapé inicial da maioria dos entusiastas dessa área, pois além de possuir uma implementação simples, também é bastante intuitivo. Quando utilizado para problemas de classificação, ele tem como objetivo classificar uma amostra de classe desconhecida com base na comparação com outros exemplos rotulados (já classificados). Ou seja, ele implementa um aprendizado baseado em instâncias, <i>Instanced Based Learning</i>. 
    <img src="img/Instance-based-Algorithms.png"> 
    <small>Fonte: https://machinelearningmastery.com/a-tour-of-machine-learning-algorithms/</small>

Como ele compara uma amostra não-rotulada com exemplos classificados (já rotulados), dizemos que ele implementa uma aprendizagem supervisionada. Ou seja, o aprendizado do algoritmo se baseia na supervisão de especialistas que anteriormente classificoram os exemplos que servirão de comparação. Assim, cada registro do meu conjunto de dados possui n atributos e uma classe que o rotula. Já a minha amostra possui apenas os atributos. O conjunto de exemplos que utilizamos para realizar as comparações é chamado de Conjunto de Treino (<i>Training Set</i>), logo cada exemplo é chamado de Instância de Treino (<i>Training Instance</i>) 

<img src="img/supervisioned_learning.png">
<small>Fonte: http://bigdata-madesimple.com/machine-learning-explained-understanding-supervised-unsupervised-and-reinforcement-learning/ </small>

### Colocar tabela que mostre uma sintese do bd que será usado, fazendo a ponte com a explicação sobre aprendizado supervisionado


Já sabemos que o KNN se baseia na comparação de uma amostra de entrada com exemplos já rotulados. Mas como ocorre esse comparação? É nesse ponto que reside a simplicidade do algoritmo. Ele somente compara amostra e um exemplo através da <b>distância euclideana</b> entre eles... 
<br>
<p style="font-size:13px">Relembrando, distância euclidiana é a distância entre dois pontos que é definida através do teorema de pitágoras. A distância euclidiana entre a amostra <i>X</i> e o exemplo <i>Y</i> é mostrada abaixo. Onde X1, X2, ..., Xn são os atributos de X e Y1, Y2, ..., Yn são os atributos Y.</p> 

\begin{align}
d(x, y) = \sqrt{ (x_1 - y_1)^2 + (x_2 - y_2)^2 + ... + (x_n - y_n)^2 }
\end{align}

Em seguida ele verifica quais são os K-vizinhos mais próximos da amostra (exemplos com as menores distâncias euclidianas). De acordo com as classes desse vizinhos podemos classificar a amostra. Sendo K=1, verificaremos apenas qual a classe do exemplo mais próximo da entrada e  a indicamos como a possível classe da amostra. Sendo k=3 verificaremos qual a classe que mais se repete nesses 3 vizinhos e a idicamos como classe da amostra. Além disso, podemos indicar essa classe em termos de probabiblidade. Por exemplo, para k=10 se 6 dos vizinhos mais próximos apresentarem a classe A, podemos informar que a amostra tem 60% de chances de pertencer a classe A.  

<img src="img/knn_ilustracao.png"> 
<small>Fonte: https://medium.com/@adi.bronshtein/a-quick-introduction-to-k-nearest-neighbors-algorithm-62214cea29c7</small>

E como podemos analisar a acurácia do nosso algoritmo? Primeiramente dividimos nosso conjunto total de registros rotulados em dois subconjuntos: o conjunto de treino (que já foi explicado) e o conjunto de treino. Em geral, selecionamos randomicamente 70% dos registros para treino e 30% para teste. 
<br><br>
Em seguida, nós aplicamos cada um dos registros de teste ao algoritmo de classificação que construímos e verificamos se a classe gerada pelo algoritmo corresponde a verdadeira classe da instância de teste. Assim, se tivermos 200 exemplos no conjunto de teste e o nosso algoritmo acertar a classe de 190 deles, podemos dizer que sua acurácia é de 95%!  

<img src="img/acuracia.png">

## Vamos Implementar! 